In [1]:
### Import packages ###
import itertools
import pandas as pd

In [2]:
def filter_job_names(df, filter_strings):
    """
    Filter DataFrame to only include rows where JobName contains any of the specified filter strings
    
    Parameters:
    -----------
    df : pandas.DataFrame
        DataFrame containing a 'JobName' column
    filter_strings : list
        List of strings to filter by
        
    Returns:
    --------
    pandas.DataFrame
        Filtered DataFrame
    """
    # Create a mask for rows that contain any of the filter strings
    mask = df['JobName'].apply(lambda x: any(filter_str in x for filter_str in filter_strings))
    
    # Return filtered DataFrame
    return df[mask]

# Input

In [3]:
AbbreviationDictionary = {"BankNote": "BN",
                          "Bar7": "B7",
                          "BreastCancer": "BC",
                          "CarEvaluation": "CE",
                          "COMPAS": "CP",
                          "FICO": "FI",
                          "Haberman": "HM",
                          "Iris": "IS",
                          "MONK1": "M1",
                          "MONK3":"M3"}

In [4]:
BatchSize = 10
DiversityWeight = 0.4

In [5]:
# Input Data Set #
Data = "BankNote" # Iris  MONK1  MONK3  Bar7 (10)  COMPAS (50) | BankNote (10)  BreastCancer (5)  CarEvaluation (10)  FICO (50)  Haberman 
JobNameAbbrev = AbbreviationDictionary[Data]

# Input directory #
dir_path = "/Users/simondn/Documents/RashomonActiveLearning/Code/raw"

# Input Parameters #
ParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                      #  "Seed":list([]),                       
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["TreeFarmsFunction"],
                       "UniqueErrorsInput": [0,1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                         # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0.045],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["medium"],                                                        # [short, medium, long, largemem, compute, cpu-g2-mem2x]
                       "Time": ["2-23:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": ["30000M"]}                                                                # [100M, 30000M, 100000M]

# Create Parameter Vector #
ParameterVector = pd.DataFrame.from_records(itertools.product(*ParameterDictionary.values()), columns=ParameterDictionary.keys())

# Include/exclude Passive Learning

In [6]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["PassiveLearning"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [1],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [0],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Include/exclude Random Forest QBC Simulations

In [7]:
### Include Random Forest ###
RandomForestParameterDictionary = {"Data":[Data],
                       "Seed":list(range(0,50)),
                       "TestProportion":[0.2],
                       "CandidateProportion":[0.8],
                       "SelectorType":["BatchQBCDiversityFunction"],
                       "ModelType":["RandomForestClassificationFunction"],
                       "UniqueErrorsInput": [0],
                       "n_estimators": [100], 
                       "regularization": [0.01],
                       "RashomonThresholdType": ["Adder"],                                                    # ["Adder", "Multiplier"]
                       "RashomonThreshold": [0],
                       "Type": ["Classification"],
                       "DiversityWeight": [DiversityWeight],
                       "BatchSize": [BatchSize],
                       "Partition": ["short"],                                                        # [short, medium, long, largemem, or compute]
                       "Time": ["00:59:00"],                                                            # [00:59:00, 11:59:00, 6-23:59:00]
                       "Memory": [1000]}                                                                # [1000, 30000, 100000]

RandomForestParameterVector = pd.DataFrame.from_records(itertools.product(*RandomForestParameterDictionary.values()), columns=RandomForestParameterDictionary.keys())

# NOTE: Comment out chunk to not include random forest simulations. ###
ParameterVector = pd.concat([ParameterVector, RandomForestParameterVector]) # NOTE: Comment out to not include random forest baseline
ParameterVector = ParameterVector.sort_values("Seed")
ParameterVector.index = range(0, ParameterVector.shape[0])

# Job and Output Name

In [8]:
### Remove Dictionary ###
del ParameterDictionary

# Generate JobName #
ParameterVector["JobName"] = (
    ParameterVector["Seed"].astype(str) +
    JobNameAbbrev + 
    "_MT" + ParameterVector["ModelType"].astype(str) +
    "_UEI" + ParameterVector["UniqueErrorsInput"].astype(str) +
    "_" + ParameterVector["RashomonThresholdType"].astype(str) + 
    ParameterVector["RashomonThreshold"].astype(str)+
    "_D" + ParameterVector["DiversityWeight"].astype(str) + 
    "B" + ParameterVector["BatchSize"].astype(str))

# Replace Job Name #
ParameterVector["JobName"] = (
    ParameterVector["JobName"]
    .str.replace(r"_MTTreeFarmsFunction_UEI0_", "_D", regex=True)
    .str.replace(r"_MTTreeFarmsFunction_UEI1_", "_U", regex=True)
    .str.replace(r"Adder", "A", regex=True)
    .str.replace(r"Multiplier", "M", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI0_", "_RF", regex=True)
    .str.replace(r"_MTRandomForestClassificationFunction_UEI1_", "_PL", regex=True)
    .str.replace(r"0.", "", regex=False)
    )

# Output Name #
ParameterVector["Output"] = ParameterVector["Data"].astype(str) + "/" + ParameterVector["ModelType"].astype(str) + "/Raw/" + ParameterVector["JobName"] + ".pkl"
ParameterVector["Output"] = ParameterVector["Output"].str.replace("Function", "", regex=False)

---

In [9]:
ParameterVector["JobName"]

0       0BN_DA045_D4B10
1        0BN_PLA0_D0B10
2        0BN_RFA0_D4B10
3       0BN_UA045_D4B10
4       1BN_DA045_D4B10
             ...       
195     48BN_PLA0_D0B10
196    49BN_DA045_D4B10
197    49BN_UA045_D4B10
198     49BN_PLA0_D0B10
199     49BN_RFA0_D4B10
Name: JobName, Length: 200, dtype: object

# Only run these ones

In [10]:
# ### Simulations that failed ###
FilterText = ['2BN_DA045', '6BN_DA045', '8BN_DA045', '18BN_DA045', '21BN_DA045', '22BN_DA045', '25BN_DA045', '39BN_DA045', '2BN_UA045', '6BN_UA045', '7BN_UA045', '8BN_UA045', '13BN_UA045', '21BN_UA045', '31BN_UA045', '33BN_UA045', '38BN_UA045', '39BN_UA045']
ParameterVector = filter_job_names(ParameterVector,FilterText)
ParameterVector.index = range(0,len(ParameterVector))
ParameterVector

,Data,Seed,TestProportion,CandidateProportion,SelectorType,ModelType,UniqueErrorsInput,n_estimators,regularization,RashomonThresholdType,RashomonThreshold,Type,DiversityWeight,BatchSize,Partition,Time,Memory,JobName,Output
0,BankNote,2,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,2BN_DA045_D4B10,BankNote/TreeFarms/Raw/2BN_DA045_D4B10.pkl
1,BankNote,2,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,2BN_UA045_D4B10,BankNote/TreeFarms/Raw/2BN_UA045_D4B10.pkl
2,BankNote,6,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,6BN_UA045_D4B10,BankNote/TreeFarms/Raw/6BN_UA045_D4B10.pkl
3,BankNote,6,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,6BN_DA045_D4B10,BankNote/TreeFarms/Raw/6BN_DA045_D4B10.pkl
4,BankNote,7,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,7BN_UA045_D4B10,BankNote/TreeFarms/Raw/7BN_UA045_D4B10.pkl
5,BankNote,8,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,8BN_UA045_D4B10,BankNote/TreeFarms/Raw/8BN_UA045_D4B10.pkl
6,BankNote,8,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,8BN_DA045_D4B10,BankNote/TreeFarms/Raw/8BN_DA045_D4B10.pkl
7,BankNote,12,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,12BN_UA045_D4B10,BankNote/TreeFarms/Raw/12BN_UA045_D4B10.pkl
8,BankNote,12,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,0,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,12BN_DA045_D4B10,BankNote/TreeFarms/Raw/12BN_DA045_D4B10.pkl
9,BankNote,13,0.2,0.8,BatchQBCDiversityFunction,TreeFarmsFunction,1,100,0.01,Adder,0.045,Classification,0.4,10,medium,2-23:59:00,30000M,13BN_UA045_D4B10,BankNote/TreeFarms/Raw/13BN_UA045_D4B10.pkl


---

# Save Parameter Vector

In [11]:
### Resave ###
output_path = "/Users/simondn/Documents/RashomonActiveLearning/Data/ParameterVectors/ParameterVector" + str(Data) + ".csv"
ParameterVector.to_csv(output_path, index=False)